In [1]:
import pandas as pd
import itertools
from yf_utils import random_slices, lookback_slices
from yf_utils import rank_perf, grp_tuples_sort_sum, top_set_sym_freq_cnt
from yf_utils import best_perf_syms_sets_lookback_slices
from myUtils import pickle_load, pickle_dump

pd.set_option("display.max_rows", 20)
pd.set_option("display.max_columns", 11)
pd.set_option("display.max_colwidth", 26)
pd.set_option("display.width", 280)


path_dir = "C:/Users/ping/MyDrive/stocks/yfinance/"
path_data_dump = path_dir + "VSCode_dump/"

fp_df_close_clean = "df_close_clean"
fp_l_days_to_drop = "l_days_to_drop"
# pickle file of dates missing from df_picks
fp_dates_missing_in_df_picks = 'dates_missing_in_df_picks'
fp_df_picks = f"df_picks"

In [2]:
df = pickle_load(path_data_dump, fp_df_picks)
df

,date_end_df_train,max_days_lookbacks,days_lookbacks,sym_freq_15,sym_freq_14,...,sym_freq_6,sym_freq_5,sym_freq_4,sym_freq_3,sym_freq_2
0,2023-12-11,120,"[30, 60, 120]",[],[],...,['GDDY'],"['ANF', 'EDU', 'LRN', ...","['CBOE', 'VRNS']","['AEO', 'AXGN', 'BTC-U...",[]
1,2023-12-11,120,"[15, 30, 60, 120]",[],[],...,"['GDDY', 'HA', 'MBI']","['ANF', 'EDU', 'LRN', ...","['AXGN', 'BPMC', 'CBOE...","['AEO', 'BTC-USD', 'BU...",[]
2,2023-12-08,120,"[30, 60, 120]",[],[],...,[],"['ANF', 'CBOE', 'EDU',...","['HIBB', 'MLKN', 'VRNS']","['BTC-USD', 'EFX', 'EX...",[]
3,2023-12-08,120,"[15, 30, 60, 120]",[],[],...,"['HA', 'MARA', 'MBI']","['ANF', 'CBOE', 'EDU',...","['AXGN', 'HIBB', 'MLKN...","['BTC-USD', 'BURL']",[]
4,2023-12-07,120,"[30, 60, 120]",[],[],...,"['GDDY', 'VRNS', 'WING']","['ANF', 'CBOE', 'EDU',...","['HIBB', 'WSM']","['BTC-USD', 'CAH', 'EX...",[]
...,...,...,...,...,...,...,...,...,...,...,...
313,2023-03-17,120,"[15, 30, 60, 120]",[],[],...,"['ANET', 'FCN', 'GBTC'...","['ACLS', 'FSLR', 'HY',...","['AMD', 'ATCO', 'MLR']",['AJRD'],[]
314,2023-03-16,120,"[30, 60, 120]",[],[],...,"['LNTH', 'SGEN', 'WST']","['ACLS', 'AMPH', 'CTLT...","['HY', 'MLR', 'OEC']","['ANET', 'ATCO', 'ATKR...",[]
315,2023-03-16,120,"[15, 30, 60, 120]",[],[],...,"['ANET', 'FCN', 'FSLR'...","['ACLS', 'AMPH', 'CTLT...",[],[],[]
316,2023-03-15,120,"[30, 60, 120]",[],[],...,"['CTLT', 'ELF', 'SGEN'...","['ACLS', 'AMPH', 'FCN'...","['ATCO', 'HY', 'LNTH',...","['ATKR', 'BURL', 'FIZZ...",[]


In [3]:
print(df)

    date_end_df_train  max_days_lookbacks     days_lookbacks sym_freq_15 sym_freq_14  ...                 sym_freq_6                 sym_freq_5                 sym_freq_4                 sym_freq_3 sym_freq_2
0          2023-12-11                 120      [30, 60, 120]          []          []  ...                   ['GDDY']  ['ANF', 'EDU', 'LRN', ...           ['CBOE', 'VRNS']  ['AEO', 'AXGN', 'BTC-U...         []
1          2023-12-11                 120  [15, 30, 60, 120]          []          []  ...      ['GDDY', 'HA', 'MBI']  ['ANF', 'EDU', 'LRN', ...  ['AXGN', 'BPMC', 'CBOE...  ['AEO', 'BTC-USD', 'BU...         []
2          2023-12-08                 120      [30, 60, 120]          []          []  ...                         []  ['ANF', 'CBOE', 'EDU',...   ['HIBB', 'MLKN', 'VRNS']  ['BTC-USD', 'EFX', 'EX...         []
3          2023-12-08                 120  [15, 30, 60, 120]          []          []  ...      ['HA', 'MARA', 'MBI']  ['ANF', 'CBOE', 'EDU',...  ['AXGN', 'HIBB', 'M

In [4]:
# Convert the "days_lookbacks" column to a list
l_days_lookbacks = df['days_lookbacks'].tolist()
l_days_lookbacks

['[30, 60, 120]',
 '[15, 30, 60, 120]',
 '[30, 60, 120]',
 '[15, 30, 60, 120]',
 '[30, 60, 120]',
 '[15, 30, 60, 120]',
 '[30, 60, 120]',
 '[15, 30, 60, 120]',
 '[30, 60, 120]',
 '[15, 30, 60, 120]',
 '[30, 60, 120]',
 '[15, 30, 60, 120]',
 '[30, 60, 120]',
 '[15, 30, 60, 120]',
 '[30, 60, 120]',
 '[15, 30, 60, 120]',
 '[30, 60, 120]',
 '[15, 30, 60, 120]',
 '[30, 60, 120]',
 '[15, 30, 60, 120]',
 '[30, 60, 120]',
 '[15, 30, 60, 120]',
 '[30, 60, 120]',
 '[15, 30, 60, 120]',
 '[30, 60, 120]',
 '[15, 30, 60, 120]',
 '[30, 60, 120]',
 '[15, 30, 60, 120]',
 '[30, 60, 120]',
 '[15, 30, 60, 120]',
 '[30, 60, 120]',
 '[15, 30, 60, 120]',
 '[30, 60, 120]',
 '[15, 30, 60, 120]',
 '[30, 60, 120]',
 '[15, 30, 60, 120]',
 '[30, 60, 120]',
 '[15, 30, 60, 120]',
 '[30, 60, 120]',
 '[15, 30, 60, 120]',
 '[30, 60, 120]',
 '[15, 30, 60, 120]',
 '[30, 60, 120]',
 '[15, 30, 60, 120]',
 '[30, 60, 120]',
 '[15, 30, 60, 120]',
 '[30, 60, 120]',
 '[15, 30, 60, 120]',
 '[30, 60, 120]',
 '[15, 30, 60, 120]',


In [5]:
l_days_lookbacks_unique = list(set(l_days_lookbacks))
l_days_lookbacks_unique

['[30, 60, 120]', '[15, 30, 60, 120]']